In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [2]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\tools")
import hsi_io as io
import hsi_decompositions as dc
print(module_path)
sys.path


D:\elena\onedrive\OneDrive - 東工大未来研情報イノベーションコア\titech\research\experiments\medHSI\src\python


['D:\\elena\\onedrive\\OneDrive - 東工大未来研情報イノベーションコア\\titech\\research\\experiments\\medHSI\\src\\python\\tests',
 'D:\\elena\\Google Drive\\titech\\research\\experiments\\macroHSI\\src\\python\\MacroHSI\\macrohsi\\tests',
 'D:\\elena\\Google Drive\\titech\\research\\experiments\\macroHSI\\src\\python\\MacroHSI\\macrohsi\\tools',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\python38.zip',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\DLLs',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\lib',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev',
 '',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\lib\\site-packages',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\lib\\site-packages\\win32',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\foxel\\anaconda3\\envs\\hsi-dev\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\foxel\\.ipython',
 'D:\\elena\\onedrive\\OneDrive - 

In [3]:
import configparser

settings_file = os.path.dirname(os.path.dirname(module_path)) + "\\conf" + "\\config.ini"
settings_file

config = configparser.ConfigParser()
config.read(settings_file, encoding = 'utf-8')
config.sections()

['Data Settings',
 'Directories',
 'Folder Names',
 'File Names',
 'Run Settings',
 'Save Settings']

In [4]:
fpath = config['Directories']['outputDir'] + "000-Datasets" + "\\hsi_normalized_full.h5"  
fpath


'D:\\elena\\mspi\\output\\000-Datasets\\hsi_normalized_full.h5'

In [5]:
f = io.load_from_h5(fpath)
list(f.keys())


['sample150',
 'sample153',
 'sample157',
 'sample160',
 'sample163',
 'sample166',
 'sample169',
 'sample172',
 'sample175',
 'sample178',
 'sample181',
 'sample184',
 'sample187',
 'sample190',
 'sample193',
 'sample196']

In [6]:
for keyz in list(f.keys()):
    print(f[keyz].shape)

(311, 158, 180)
(311, 198, 324)
(311, 132, 108)
(311, 120, 164)
(311, 72, 80)
(311, 244, 308)
(311, 208, 224)
(311, 244, 212)
(311, 234, 212)
(311, 258, 424)
(311, 342, 332)
(311, 282, 436)
(311, 376, 236)
(311, 232, 428)
(311, 392, 236)
(311, 126, 128)


In [12]:
#dataz =  io.load_from_h5(fpath)
dataList = io.load_dataset(fpath, 'image')
print("images in dataset", len(dataList))
print("datatype ", (dataList[0]).dtype)

#how to handle data with different size 

images in dataset 16
datatype  float64


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(dataList, test_size=0.1, random_state=42)
print(X_train.shape, X_test.shape)

AttributeError: 'list' object has no attribute 'shape'